In [26]:
import pandas as pd
import os
import datetime
import pytz
import numpy as np
import pickle

In [27]:
# read the txt readme files

demo=pd.DataFrame()

numbers=[2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]

for i in numbers:
    filename = 'data\WESAD\S%i\S%i_readme.txt'%(i,i)

#filename = 'data\WESAD\S8\S8_readme.txt'

    with open(filename, 'r') as file:
        lines = file.readlines()

    subject_number = int(os.path.basename(filename).split('S')[1].split('_')[0])
    data = {'id': subject_number}
    current_header = None

    for line in lines:
        parts = line.strip().split(':')  # Split using ':' as delimiter
        if len(parts) == 1:  # If ':' is not found, split using '?' as delimiter
            parts = line.strip().split('?')
        if len(parts) > 1:
            header = parts[0].strip()  # Extract the column header
            value = ':'.join(parts[1:]).strip()  # Join remaining parts as value
            data[header] = value

    # Create a Pandas DataFrame
    user_df = pd.DataFrame([data])
    # before the next user, add this user_df to the total dataframe
    demo = pd.concat([demo, user_df])

demo = demo.drop(columns=['Stress condition / TSST interview part'])
demo

,id,Age,Height (cm),Weight (kg),Gender,Dominant hand,Did you drink coffee today,Did you drink coffee within the last hour,Did you do any sports today,Are you a smoker,Did you smoke within the last hour,Do you feel ill today
0,2,27,175,80,male,right,NO,NO,NO,NO,NO,NO
0,3,27,173,69,male,right,NO,NO,NO,NO,NO,NO
0,4,25,175,90,male,right,NO,NO,NO,NO,NO,NO
0,5,35,189,80,male,right,YES,NO,NO,NO,NO,NO
0,6,27,170,66,male,right,YES,NO,NO,YES,NO,NO
0,7,28,184,74,male,right,NO,NO,YES,NO,NO,NO
0,8,27,172,64,female,left,YES,NO,YES,NO,NO,NO
0,9,26,181,75,male,right,NO,NO,NO,NO,NO,YES
0,10,28,178,76,male,right,NO,NO,NO,NO,NO,NO
0,11,26,171,54,female,right,YES,NO,NO,NO,NO,NO


In [28]:
# read the PANAS self questionnaires csv files

# PANAS scores can range from 10 to 50, with higher scores representing higher levels of positive or negative affect, respectively.

def calculate_panas_scores_from_dataframe(df):

    positive_headers = ["Interested", "Inspired", "Guilty", "Excited", "Irritable", "Alert", "Attentive", "Jittery", "Afraid", "Stressed"]
    negative_headers = ["Active", "Distressed", "Annoyed", "Strong", "Scared", "Hostile", "Proud", "Enthusiastic", "Ashamed", "Sad", "(Angry)", "(Irritated)"]

    df[positive_headers + negative_headers] = df[positive_headers + negative_headers].apply(pd.to_numeric, errors='coerce').fillna(0)

    pa_score = df[positive_headers].astype(int).sum(axis=1)
    na_score = df[negative_headers].astype(int).sum(axis=1)

    df['PA_Score'] = pa_score
    df['NA_Score'] = na_score

    return df

numbers=[2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]

panas=pd.DataFrame()

for user in numbers:
    filename = 'data\WESAD\S%i\S%i_quest.csv'%(i,i)
    #user_df=pd.DataFrame()

    lines_to_read = 5
    data = []

    header_mapping = {
    1: "Active",
    2: "Distressed",
    3: "Interested",
    4: "Inspired",
    5: "Annoyed",
    6: "Strong",
    7: "Guilty",
    8: "Scared",
    9: "Hostile",
    10: "Excited",
    11: "Proud",
    12: "Irritable",
    13: "Enthusiastic",
    14: "Ashamed",
    15: "Alert",
    16: "Nervous",
    17: "Determined",
    18: "Attentive",
    19: "Jittery",
    20: "Afraid",
    21: "Stressed",
    22: "Frustrated",
    23: "Happy",
    24: "Sad",
    25: "(Angry)",
    26: "(Irritated)"
}

    with open(filename, 'r') as csvfile:
        lines_read = 0
        for line in csvfile:
            if lines_read >= lines_to_read:
                break
            if line.strip().startswith('# PANAS'):
                values = line.strip().split(';')[1:]
                data.append(values)
                lines_read += 1

    user_df = pd.DataFrame(data, columns=[header_mapping[i] for i in range(1, len(data[0]) + 1)])
    user_df['id'] = user

    # before the next user, add this user_df to the total dataframe
    panas = pd.concat([panas, user_df])

panas = calculate_panas_scores_from_dataframe(panas)
panas = panas.drop(panas.iloc[:, 0:26],axis = 1)

panas

,id,PA_Score,NA_Score
0,2,20,12
1,2,24,15
2,2,15,10
3,2,31,38
4,2,23,14
...,...,...,...
0,17,20,12
1,17,24,15
2,17,15,10
3,17,31,38


In [29]:
# read the STAI self questionnaires csv files

# to calculate the total stress score simply sum per row

numbers=[2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]

stai=pd.DataFrame()

for user in numbers:
    filename = 'data\WESAD\S%i\S%i_quest.csv'%(user,user)

    lines_to_read = 5
    data = []

    with open(filename, 'r') as csvfile:
        lines_read = 0
        for line in csvfile:
            if lines_read >= lines_to_read:
                break
            if line.strip().startswith('# STAI'):
                values = line.strip().split(';')[1:]
                data.append(values)
                lines_read += 1

    user_df = pd.DataFrame(data)
    user_df = user_df.iloc[:, 0:6]
    user_df['id'] = user

    # before the next user, add this user_df to the total dataframe
    stai = pd.concat([stai, user_df])

for column in stai.columns:
    stai[column] = pd.to_numeric(stai[column], errors='coerce')


stai['stai_stress'] = stai.iloc[:, 0:6].sum(axis=1)
stai = stai.iloc[:, 6:]

mean_stai = stai['stai_stress'].mean()
std_stai = stai['stai_stress'].std()

def get_stai_category(score):
    if score < mean_stai-0.5*std_stai:
        return "Below average"
    if score > mean_stai+0.5*std_stai:
        return "Above average"
    return "Average"

stai['stai_stress_category'] = stai['stai_stress'].apply(lambda score: get_stai_category(score))

stai = stai.drop(columns=['stai_stress'])

stai

,id,stai_stress_category
0,2,Above average
1,2,Below average
2,2,Above average
3,2,Average
4,2,Above average
...,...,...
0,17,Below average
1,17,Above average
2,17,Below average
3,17,Average


In [30]:
def adds_date_column(df):
    timestamp_utc = df.iloc[0, 0]
    datetime_utc = datetime.datetime.utcfromtimestamp(timestamp_utc)
    datetime_utc = datetime_utc.replace(tzinfo=pytz.utc)
    target_timezone = pytz.timezone('America/New_York')
    datetime_local = datetime_utc.astimezone(target_timezone)
    df['Date'] = datetime_local.strftime('%Y-%m-%d %H:%M:%S %Z')
    e4_acc = e4_acc.iloc[1:]
    starting_date = e4_acc.iloc[0,3]
    num_rows = e4_acc.shape[0]
    date_range = pd.date_range(start=starting_date, periods=num_rows, freq='S')
    e4_acc['date'] = pd.DataFrame({'date': date_range})
    e4_acc = e4_acc.drop(columns="Date")



# read E4 data
e4_acc = pd.read_csv('data\WESAD\S2\S2_E4_Data\ACC.csv', header=None)
timestamp_utc = e4_acc.iloc[0, 0]
datetime_utc = datetime.datetime.utcfromtimestamp(timestamp_utc)
datetime_utc = datetime_utc.replace(tzinfo=pytz.utc)
target_timezone = pytz.timezone('America/New_York')
datetime_local = datetime_utc.astimezone(target_timezone)
e4_acc['Date'] = datetime_local.strftime('%Y-%m-%d %H:%M:%S %Z')
e4_acc = e4_acc.iloc[1:]
starting_date = e4_acc.iloc[0,3]
num_rows = e4_acc.shape[0]
date_range = pd.date_range(start=starting_date, periods=num_rows, freq='S')
e4_acc['date'] = pd.DataFrame({'date': date_range})
e4_acc = e4_acc.drop(columns="Date")
e4 = pd.DataFrame()
e4 = pd.concat([e4, e4_acc])
e4

C:\Users\Christina\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,0,1,2,date
1,32.0,32.0,32.0,2017-05-22 03:15:26
2,29.0,-6.0,55.0,2017-05-22 03:15:27
3,28.0,-6.0,55.0,2017-05-22 03:15:28
4,28.0,-6.0,55.0,2017-05-22 03:15:29
5,28.0,-6.0,55.0,2017-05-22 03:15:30
...,...,...,...,...
251967,61.0,-14.0,3.0,2017-05-25 01:14:52
251968,61.0,-14.0,3.0,2017-05-25 01:14:53
251969,60.0,-14.0,4.0,2017-05-25 01:14:54
251970,63.0,-14.0,5.0,2017-05-25 01:14:55


In [20]:
e4_eda = pd.read_csv('data\WESAD\S2\S2_E4_Data\EDA.csv', header=None, skiprows=1)
e4_eda = e4_eda.rename(columns={e4_eda.columns[0]: 'EDA'})
e4_eda

,EDA
0,4.000000
1,0.000000
2,0.328021
3,0.410026
4,0.433090
...,...
31490,0.409011
31491,0.397480
31492,0.389792
31493,0.387229


In [21]:
e4_bvp = pd.read_csv('data\WESAD\S2\S2_E4_Data\BVP.csv', header=None, skiprows=1)
e4_bvp = e4_bvp.rename(columns={e4_bvp.columns[0]: 'BVP'})
e4_bvp

,BVP
0,64.00
1,-0.00
2,-0.00
3,-0.00
4,-0.00
...,...
503939,96.51
503940,92.92
503941,85.61
503942,75.98
